In [75]:
import pandas as pd
import math
from datetime import datetime
from datetime import timedelta
import time

In [76]:
df = pd.read_csv('./temp/jobtable.csv')
df

,JOBID,JOBNAME,UNLOADED,LOAD_DAY,LOAD_TIME,UNLOAD_TIME,FREQ,NEXT_RUN_TIME
0,1,sample_stage,N,"1,2,3,4,5",19:00,20:00,120,NaN


In [92]:
def scanJobTable():
    dbcol = { 'JOBID':0, 'JOBNAME':1, 'UNLOADED':2, 'LOAD_DAY':3, 'LOAD_TIME':4, 'UNLOAD_TIME':5, 'FREQ':6, 'NEXT_RUN_TIME':7 }
    
    df = pd.read_csv('../../../temp/jobtable.csv')
    now = datetime.now()
    for row in df.values:
        loaded_ok = row[dbcol['UNLOADED']] != 'Y'
        
        load_time = datetime.strptime(row[dbcol['LOAD_TIME']], '%H:%M').time()
        unload_time = datetime.strptime(row[dbcol['UNLOAD_TIME']], '%H:%M').time()
        time_ok = load_time <= now.time() and now.time() <= unload_time

        load_days = row[dbcol['LOAD_DAY']].split(',')
        day_ok = str(now.weekday() + 1) in load_days

        next_run_ok = row[dbcol['NEXT_RUN_TIME']] is None or math.isnan(row[dbcol['NEXT_RUN_TIME']])
        if not next_run_ok:
            next_run_time = datetime.strptime(row[dbcol['NEXT_RUN_TIME']], '%H:%M').time()
            next_run_ok = next_run_time.time() <= now.time()

        print('load:{}, time:{}, day:{}, next:{}'.format(loaded_ok, time_ok, day_ok, next_run_ok))
        if loaded_ok and time_ok and day_ok and next_run_ok:
            print('>>>> RUNNING job {}'.format(row[dbcol['JOBNAME']]))
            #runJob(row[dbcol['JOBNAME']], row[dbcol['FREQ']])
            
scanJobTable()

FileNotFoundError: File b'../../../temp/jobtable.csv' does not exist

In [82]:
now = datetime.now()
for row in df.values:
    loaded_ok = row[2] != 'Y'
    
    load_time = datetime.strptime(row[4], '%H:%M').time()
    unload_time = datetime.strptime(row[5], '%H:%M').time()
    time_ok = load_time <= now.time() and now.time() <= unload_time
    
    load_days = row[3].split(',')
    day_ok = str(now.weekday() + 1) in load_days
    
    next_run_ok = row[7] is None or math.isnan(row[7])
    if not next_run_ok:
        next_run_time = datetime.strptime(row[7], '%H:%M').time()
        next_run_ok = next_run_time.time() <= now.time()

    if loaded_ok and time_ok and day_ok and next_run_ok:
        runJob(row[1], row[6])

In [72]:
def runJob(jobname, freq):
    # TODO run job
    
    
    if freq is None or math.isnan(freq):
        # TODO, no frequency, set UNLOADED to Y        
        a = "UPDATE db SET UNLOADED = 'Y'"
    nextrun = datetime.now() + timedelta(seconds=freq)
    # TODO set next run in database